In [1]:
import graphlab as gl
import datetime
from dateutil import parser as datetime_parser

2016-04-19 14:02:12,212 [INFO] graphlab.deploy._session, 584: Using session dir: /Users/zach/graphlab-dev2/local_scripts/../artifacts


# Load Data

In [2]:
# Table of product purchases
purchases = gl.SFrame.read_csv('dataset/online_retail.csv')

This commercial license of GraphLab Create is assigned to engr@turi.com.


2016-04-19 14:02:15,541 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1461099733.log


Finished parsing file /Users/zach/sample-churn-predictor/dataset/online_retail.csv

Parsing completed. Parsed 100 lines in 8.38356 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str,int,str,float,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/zach/sample-churn-predictor/dataset/online_retail.csv

Parsing completed. Parsed 541909 lines in 4.73211 secs.

# Prepare Data

Convert the datetime strings to Python datetimes and create a GraphLab Create TimeSeries
from the `InvoiceDate` column.

In [3]:
# Convert InvoiceDate strings (e.g. "12/1/10 8:26") to datetimes
purchases['InvoiceDate'] = purchases['InvoiceDate'].apply(datetime_parser.parse)

# Create a TimeSeries
timeseries = gl.TimeSeries(purchases, 'InvoiceDate')

# Train Churn Predictor Model

A churn forecast requires a time boundary and a churn period.
Activity before the boundary is used to train the model.
After the boundary, activity (or lack of activity)
during the churn period is used to define whether the
user churned.

In [4]:
# Split the data into train and validation
train, valid = gl.churn_predictor.random_split(timeseries, user_id='CustomerID', fraction=0.8, seed = 1)

# Train the model using data before August
churn_boundary_oct = datetime.datetime(year = 2011, month = 8, day = 1)
# Define churn as "inactive for 30 days after August 1st 2011"
churn_period = datetime.timedelta(days = 30)

model = gl.churn_predictor.create(train, user_id='CustomerID',
                                  features = ['Quantity'],
                                  churn_period = churn_period,
                                  time_boundaries = [churn_boundary_oct])

PROGRESS: Grouping observation_data by user.
PROGRESS: Resampling grouped observation_data by time-period 1 day, 0:00:00.
PROGRESS: Generating features for time-boundary.
PROGRESS: --------------------------------------------------
PROGRESS: Features for 2011-07-31 17:00:00.
PROGRESS: Training a classifier model.


Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 2555

Number of classes           : 2

...

| 4         | 0.113640     | 0.858317          | 0.424838          |

| 5         | 0.136425     | 0.858708          | 0.404621          |

| 6         | 0.163361     | 0.861448          | 0.391001          |

+-----------+--------------+-------------------+-------------------+

PROGRESS: --------------------------------------------------
PROGRESS: Model training complete: Next steps
PROGRESS: --------------------------------------------------
PROGRESS: (1) Evaluate the model at various timestamps in the past:
PROGRESS:       metrics = model.evaluate(data, time_in_past)
PROGRESS: (2) Make a churn forecast for a timestamp in the future:
PROGRESS:       predictions = model.predict(data, time_in_future)


# Explore and Evaluate the Model

In [5]:
# Interactively explore churn predictions
view = model.views.overview(exploration_set=timeseries,
                            validation_set=valid,
                            exploration_time=churn_boundary_oct,
                            validation_time=churn_boundary_oct)
view.show()

2016-04-19 14:04:16,633 [WARNING] graphlab.toolkits.churn_predictor._churn_predictor, 1881: This feature is currently in beta. Please use with caution and not in mission-critical applications. For feedback or suggestions on this feature, please e-mail product-feedback@turi.com.


PROGRESS: Making a churn forecast for the time window:
PROGRESS: --------------------------------------------------
PROGRESS:  Start : 2011-08-01 00:00:00
PROGRESS:  End   : 2011-08-31 00:00:00
PROGRESS: --------------------------------------------------
PROGRESS: Grouping dataset by user.
PROGRESS: Resampling grouped observation_data by time-period 1 day, 0:00:00.
PROGRESS: Generating features for boundary 2011-08-01 00:00:00.
PROGRESS: Not enough data to make predictions for 1170 user(s). 
PROGRESS: Making a churn forecast for the time window:
PROGRESS: --------------------------------------------------
PROGRESS:  Start : 2011-08-01 00:00:00
PROGRESS:  End   : 2011-08-31 00:00:00
PROGRESS: --------------------------------------------------
PROGRESS: Grouping dataset by user.
PROGRESS: Resampling grouped observation_data by time-period 1 day, 0:00:00.
PROGRESS: Generating features for boundary 2011-08-01 00:00:00.
PROGRESS: Not enough data to make predictions for 254 user(s). 
PROGRES